# Optimizer Evaluation Across Model Architectures

This notebook provides a comprehensive analysis of different optimizers across various model architectures for our machine learning project. We'll evaluate performance metrics, convergence rates, and overall model quality.

## Table of Contents

1. [Setup and Configuration](#setup-and-configuration)
2. [Model Architectures Overview](#model-architectures-overview)
3. [Optimizer Definitions](#optimizer-definitions)
4. [Training Automation](#training-automation)
5. [Performance Visualization](#performance-visualization)
6. [Optimizer Comparison](#optimizer-comparison)
7. [Best Optimizer Analysis](#best-optimizer-analysis)
8. [Conclusions and Recommendations](#conclusions-and-recommendations)

## Setup and Configuration

Let's import the necessary libraries and set up our environment for the optimizer evaluation experiments. We'll configure paths, import required modules, and define visualization settings.

In [4]:
import os
import sys
import json
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML

# Add parent directory to path to import project modules
# This ensures that imports from 'src' will work correctly
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Set plotting style
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Configure matplotlib for better visualization
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.dpi'] = 150

# Configuration parameters
PROJECT_ROOT = Path(os.path.abspath(os.path.join(os.getcwd(), '../..')))
RESULTS_DIR = PROJECT_ROOT / 'logs'
MAIN_SCRIPT = PROJECT_ROOT / 'main.py'

# Define models and optimizers to evaluate
MODELS = ['Base', 'Wide', 'Advanced']
OPTIMIZERS = ['Adam', 'ImprovedAdam', 'Nadam', 'RMSprop', 'SGD', 'Adadelta']
EPOCHS = 15  # Use fewer epochs for faster evaluation
BATCH_SIZE = 32

print(f"Project Root: {PROJECT_ROOT}")
print(f"Results Directory: {RESULTS_DIR}")
print(f"Models to evaluate: {MODELS}")
print(f"Optimizers to evaluate: {OPTIMIZERS}")

Project Root: /Users/marcofurrer/Documents/github/dspro2
Results Directory: /Users/marcofurrer/Documents/github/dspro2/logs
Models to evaluate: ['Base', 'Wide', 'Advanced']
Optimizers to evaluate: ['Adam', 'ImprovedAdam', 'Nadam', 'RMSprop', 'SGD', 'Adadelta']


## Model Architectures Overview

In this section, we'll examine the different model architectures available in our project. We'll load the architectures from our code base and provide a summary of each.

In [3]:
# Import model architectures directly from their module files
try:
    # Import all possible models
    from src.models.Deep import model as deep_model
    from src.models.CorrelationModel import model as correlation_model
    from src.models.ImprovedModel import model as improved_model
    from src.models.BestModel import model as best_model
    from src.models.Residual import model as residual_model
    from src.models.Advanced import model as advanced_model
    from src.models.Wide import model as wide_model
    from src.models.Base import model as base_model
    
    # Create a lookup dictionary of imported models
    available_models = {
        'Deep': deep_model,
        'Correlation': correlation_model,
        'Improved': improved_model,
        'Best': best_model,
        'Residual': residual_model,
        'Advanced': advanced_model,
        'Wide': wide_model,
        'Base': base_model
    }
    
    # Display summaries for selected models only
    for model_name in MODELS:
        if model_name in available_models:
            print(f"\n\n{'='*50}")
            print(f"Model Architecture: {model_name}")
            print(f"{'='*50}")
            available_models[model_name].summary()
        else:
            print(f"Warning: Model '{model_name}' not found in available models")
            
except Exception as e:
    print(f"Error loading model architectures: {e}")
    print("Will continue with experiment configuration without model summaries.")

Error loading model architectures: No module named 'src'
Will continue with experiment configuration without model summaries.


## Optimizer Definitions

Let's define the optimizers we want to evaluate and their configurations.

In [ ]:
# Import the optimizers
try:
    from tensorflow.keras.optimizers import (
        Adam, SGD, RMSprop, Nadam, Adagrad, Adadelta, Adamax
    )
    
    # Try importing custom optimizers if available
    try:
        from src.optimizers.Adam import optimizer as adam_optimizer
        from src.optimizers.ImprovedAdam import optimizer as improved_adam_optimizer
        from src.optimizers.Nadam import optimizer as nadam_optimizer
        from src.optimizers.RMSprop import optimizer as rmsprop_optimizer
        from src.optimizers.SGD import optimizer as sgd_optimizer
        from src.optimizers.Adadelta import optimizer as adadelta_optimizer
        from src.optimizers.Adagrad import optimizer as adagrad_optimizer
        from src.optimizers.Adamax import optimizer as adamax_optimizer
        custom_optimizers = True
    except ImportError:
        print("Using standard TensorFlow optimizers as custom optimizers not found")
        custom_optimizers = False
    
    # Create a dict of optimizers
    if custom_optimizers:
        optimizers = {
            'Adam': adam_optimizer,
            'ImprovedAdam': improved_adam_optimizer,
            'Nadam': nadam_optimizer,
            'RMSprop': rmsprop_optimizer,
            'SGD': sgd_optimizer,
            'Adadelta': adadelta_optimizer,
            'Adagrad': adagrad_optimizer,
            'Adamax': adamax_optimizer
        }
    else:
        # Create standard optimizers with default configs
        optimizers = {
            'Adam': Adam(),
            'SGD': SGD(),
            'RMSprop': RMSprop(),
            'Nadam': Nadam(),
            'Adagrad': Adagrad(),
            'Adadelta': Adadelta(),
            'Adamax': Adamax()
        }
        
    # Print optimizer configurations
    for name, optimizer in optimizers.items():
        if name in OPTIMIZERS:  # Only show selected optimizers
            print(f"Optimizer: {name}")
            print(f"Configuration: {optimizer.get_config()}")
            print("-" * 50)
except Exception as e:
    print(f"Error loading optimizers: {e}")
    print("Will continue with experiment configuration without optimizer details.")

## Training Automation

In this section, we'll create functions to automate the training process across different model architectures and optimizers.

In [ ]:
def train_model(model_name, optimizer_name, epochs=EPOCHS, batch_size=BATCH_SIZE):
    """
    Trains a model with specified architecture and optimizer by calling main.py
    
    Args:
        model_name (str): Name of the model architecture to use
        optimizer_name (str): Name of the optimizer to use
        epochs (int): Number of epochs for training
        batch_size (int): Batch size for training
        
    Returns:
        str: Path to the results directory
    """
    # Generate a unique run ID
    run_id = f"{model_name}_{optimizer_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # Command to run main.py with appropriate arguments
    cmd = [
        "python", str(MAIN_SCRIPT),
        "--model", model_name,
        "--optimizer", optimizer_name,
        "--epochs", str(epochs),
        "--batch_size", str(batch_size),
        "--run_id", run_id,
        "--verbose", "1"
    ]
    
    print(f"Running: {' '.join(cmd)}")
    
    # Execute the command
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Print the result
    print(result.stdout)
    if result.stderr:
        print(f"Error: {result.stderr}")
    
    # Return the path to the results directory
    return str(RESULTS_DIR / run_id)

def run_all_experiments(model_names, optimizer_names, epochs=EPOCHS, batch_size=BATCH_SIZE):
    """
    Runs all combinations of models and optimizers
    
    Args:
        model_names (list): List of model architecture names
        optimizer_names (list): List of optimizer names
        epochs (int): Number of epochs for training
        batch_size (int): Batch size for training
        
    Returns:
        dict: Dictionary mapping experiment combinations to result paths
    """
    results = {}
    total_experiments = len(model_names) * len(optimizer_names)
    counter = 1
    
    for model_name in model_names:
        for optimizer_name in optimizer_names:
            print(f"\nExperiment {counter}/{total_experiments}: {model_name} with {optimizer_name}")
            result_path = train_model(model_name, optimizer_name, epochs, batch_size)
            results[(model_name, optimizer_name)] = result_path
            counter += 1
            
    return results

Let's run a set of experiments with selected models and optimizers. You can adjust these lists to include the specific combinations you want to evaluate.

In [ ]:
# Select a subset of models and optimizers for evaluation to reduce computation time
# You can uncomment the full lists for a comprehensive evaluation
selected_models = ['Advanced', 'Residual', 'Best']
selected_optimizers = ['Adam', 'ImprovedAdam', 'Nadam', 'RMSprop', 'SGD']

# Uncomment to run the experiments
# experiment_results = run_all_experiments(
#     model_names=selected_models,
#     optimizer_names=selected_optimizers,
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE
# )
# 
# # Print the results directory paths
# for (model, optimizer), result_path in experiment_results.items():
#     print(f"{model} + {optimizer}: {result_path}")

# For notebook development, create a simulated results dict
# In practice, you would use the actual experiment_results from above
experiment_results = {}
for model in selected_models:
    for optimizer in selected_optimizers:
        experiment_results[(model, optimizer)] = f"{RESULTS_DIR}/{model}_{optimizer}_20230101_000000"

print("Experiment configuration complete. Ready to run experiments or analyze existing results.")

## Performance Visualization

Now that we've trained models with different optimizers (or have existing results), let's visualize their performance.

First, let's create functions to load and process the results.

# Import the optimizers
try:
    # Import the standard TensorFlow optimizers as fallback
    from tensorflow.keras.optimizers import (
        Adam, SGD, RMSprop, Nadam, Adagrad, Adadelta, Adamax
    )
    
    # Try importing custom optimizers from the project
    # The sys.path.append in the setup ensures these imports work correctly
    try:
        # Import optimizers from the project's source directory
        from src.optimizers.Adam import optimizer as adam_optimizer
        from src.optimizers.ImprovedAdam import optimizer as improved_adam_optimizer
        from src.optimizers.Nadam import optimizer as nadam_optimizer
        from src.optimizers.RMSprop import optimizer as rmsprop_optimizer
        from src.optimizers.SGD import optimizer as sgd_optimizer
        from src.optimizers.Adadelta import optimizer as adadelta_optimizer
        from src.optimizers.Adagrad import optimizer as adagrad_optimizer
        from src.optimizers.Adamax import optimizer as adamax_optimizer
        
        # If we get here, custom optimizers were found
        custom_optimizers = True
        print("Successfully imported custom optimizers from src.optimizers")
    except ImportError as e:
        print(f"Custom optimizers not found: {e}")
        print("Using standard TensorFlow optimizers instead")
        custom_optimizers = False
    
    # Create a dictionary of optimizers based on availability
    if custom_optimizers:
        optimizers = {
            'Adam': adam_optimizer,
            'ImprovedAdam': improved_adam_optimizer,
            'Nadam': nadam_optimizer,
            'RMSprop': rmsprop_optimizer,
            'SGD': sgd_optimizer,
            'Adadelta': adadelta_optimizer,
            'Adagrad': adagrad_optimizer,
            'Adamax': adamax_optimizer
        }
    else:
        # Create standard optimizers with default configs as fallback
        optimizers = {
            'Adam': Adam(),
            'SGD': SGD(),
            'RMSprop': RMSprop(),
            'Nadam': Nadam(),
            'Adagrad': Adagrad(),
            'Adadelta': Adadelta(),
            'Adamax': Adamax()
        }
        
    # Print optimizer configurations for selected optimizers
    for name, optimizer in optimizers.items():
        if name in OPTIMIZERS:  # Only show selected optimizers
            print(f"Optimizer: {name}")
            print(f"Configuration: {optimizer.get_config()}")
            print("-" * 50)
except Exception as e:
    print(f"Error loading optimizers: {e}")
    print("Will continue with experiment configuration without optimizer details.")

In [ ]:
def load_experiment_results(experiment_results):
    """
    Loads metrics from all experiments
    
    Args:
        experiment_results (dict): Dictionary mapping experiment combinations to result paths
        
    Returns:
        pandas.DataFrame: DataFrame containing metrics for all experiments
    """
    all_metrics = []
    
    for (model, optimizer), result_path in experiment_results.items():
        # Look for metrics.csv file
        metrics_file = Path(result_path) / "metrics.csv"
        
        if metrics_file.exists():
            # Load the metrics
            df = pd.read_csv(metrics_file)
            
            # Add model and optimizer columns
            df['model'] = model
            df['optimizer'] = optimizer
            
            all_metrics.append(df)
        else:
            print(f"Warning: No metrics file found for {model} with {optimizer} at {metrics_file}")
            
            # For demonstration, create simulated metrics if the file doesn't exist
            # In practice, this would be removed and you'd use only real data
            epochs = 10
            simulated_metrics = {
                'epoch': list(range(1, epochs+1)),
                'loss': np.random.rand(epochs) * 0.5 + 0.2,
                'accuracy': np.random.rand(epochs) * 0.3 + 0.6,
                'val_loss': np.random.rand(epochs) * 0.6 + 0.3,
                'val_accuracy': np.random.rand(epochs) * 0.3 + 0.5,
                'time_per_epoch': np.random.rand(epochs) * 2 + 1,
                'model': [model] * epochs,
                'optimizer': [optimizer] * epochs
            }
            simulated_df = pd.DataFrame(simulated_metrics)
            
            # Ensure the metrics follow a reasonable learning curve
            # Loss should generally decrease
            simulated_df['loss'] = sorted(simulated_df['loss'], reverse=True)
            simulated_df['val_loss'] = sorted(simulated_df['val_loss'], reverse=True) 
            
            # Accuracy should generally increase
            simulated_df['accuracy'] = sorted(simulated_df['accuracy'])
            simulated_df['val_accuracy'] = sorted(simulated_df['val_accuracy'])
            
            # Add some noise to make it realistic
            for col in ['loss', 'accuracy', 'val_loss', 'val_accuracy']:
                noise = np.random.normal(0, 0.01, epochs)
                simulated_df[col] += noise
                
            all_metrics.append(simulated_df)
    
    if all_metrics:
        return pd.concat(all_metrics, ignore_index=True)
    else:
        return pd.DataFrame()


def get_summary_metrics(metrics_df):
    """
    Computes summary metrics across epochs for each model-optimizer combination
    
    Args:
        metrics_df (pandas.DataFrame): DataFrame with training metrics
        
    Returns:
        pandas.DataFrame: DataFrame with summary metrics
    """
    if metrics_df.empty:
        return pd.DataFrame()
    
    # Group by model and optimizer
    grouped = metrics_df.groupby(['model', 'optimizer'])
    
    # Compute summary metrics
    summary = grouped.agg(
        final_val_loss=('val_loss', 'last'),
        min_val_loss=('val_loss', 'min'),
        final_val_accuracy=('val_accuracy', 'last'),
        max_val_accuracy=('val_accuracy', 'max'),
        convergence_epoch=('val_loss', lambda x: x.argmin() + 1),
        training_time=('time_per_epoch', 'sum')
    ).reset_index()
    
    return summary

In [ ]:
# Load results
all_metrics = load_experiment_results(experiment_results)

if not all_metrics.empty:
    # Display the first few rows
    display(all_metrics.head())
    
    # Get summary metrics
    summary_metrics = get_summary_metrics(all_metrics)
    display(summary_metrics)
else:
    print("No metrics data found. Please check experiment results.")

Now let's create comprehensive visualizations to compare optimizers across model architectures.

In [ ]:
def plot_learning_curves(metrics_df, metric='val_loss'):
    """
    Plots learning curves for each model-optimizer combination
    
    Args:
        metrics_df (pandas.DataFrame): DataFrame with training metrics
        metric (str): Metric to plot ('val_loss', 'val_accuracy', etc.)
    """
    if metrics_df.empty:
        print("No metrics data available for plotting.")
        return
    
    # Get unique models and optimizers
    models = metrics_df['model'].unique()
    optimizers = metrics_df['optimizer'].unique()
    
    # Create subplots - one for each model
    fig, axes = plt.subplots(len(models), 1, figsize=(14, 5 * len(models)), sharex=True)
    if len(models) == 1:
        axes = [axes]  # Handle case with only one model
        
    # Define line styles and colors for optimizers
    colors = plt.cm.tab10(np.linspace(0, 1, len(optimizers)))
    
    for i, model in enumerate(models):
        ax = axes[i]
        
        for j, optimizer in enumerate(optimizers):
            # Filter data for this model and optimizer
            data = metrics_df[(metrics_df['model'] == model) & (metrics_df['optimizer'] == optimizer)]
            
            if not data.empty:
                # Plot the learning curve
                ax.plot(data['epoch'], data[metric], 
                        label=optimizer, 
                        color=colors[j],
                        marker='o', 
                        markersize=4, 
                        linewidth=2)
        
        # Set labels and title
        ax.set_title(f"{model} - {metric.replace('_', ' ').title()} over Epochs")
        ax.set_ylabel(metric.replace('_', ' ').title())
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend(title="Optimizer")
        
    # Set common x-label
    fig.text(0.5, 0.04, 'Epoch', ha='center', va='center', fontsize=14)
    
    plt.tight_layout(pad=3.0)
    plt.show()

def create_heatmap(summary_df, metric_col, title):
    """
    Creates a heatmap for a specific metric across model-optimizer combinations
    
    Args:
        summary_df (pandas.DataFrame): DataFrame with summary metrics
        metric_col (str): Column name for the metric to visualize
        title (str): Title for the heatmap
    """
    if summary_df.empty:
        print("No summary data available for heatmap.")
        return
    
    # Pivot the DataFrame to get models as rows and optimizers as columns
    pivot_df = summary_df.pivot(index='model', columns='optimizer', values=metric_col)
    
    # Create the heatmap
    plt.figure(figsize=(12, 8))
    
    # Determine color map based on metric (lower is better for loss, higher is better for accuracy)
    cmap = "YlOrRd_r" if "loss" in metric_col else "YlGn"
    
    sns.heatmap(pivot_df, annot=True, cmap=cmap, fmt=".4f", linewidths=.5)
    plt.title(title)
    plt.ylabel('Model Architecture')
    plt.xlabel('Optimizer')
    plt.tight_layout()
    plt.show()

def create_radar_chart(summary_df, optimizers, metrics):
    """
    Creates a radar chart comparing optimizers across multiple metrics
    
    Args:
        summary_df (pandas.DataFrame): DataFrame with summary metrics
        optimizers (list): List of optimizers to include
        metrics (list): List of metrics to compare
    """
    if summary_df.empty:
        print("No summary data available for radar chart.")
        return
    
    # Get unique models
    models = summary_df['model'].unique()
    
    # Create subplots - one for each model
    fig = plt.figure(figsize=(16, 4 * len(models)))
    
    for i, model in enumerate(models):
        # Filter data for this model
        model_data = summary_df[summary_df['model'] == model]
        
        # Create a subplot
        ax = fig.add_subplot(len(models), 1, i+1, polar=True)
        
        # Number of metrics
        num_metrics = len(metrics)
        angles = np.linspace(0, 2*np.pi, num_metrics, endpoint=False).tolist()
        angles += angles[:1]  # Close the loop
        
        # Set the labels for each axis
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels([m.replace('_', ' ').title() for m in metrics])
        
        # Colors for different optimizers
        colors = plt.cm.tab10(np.linspace(0, 1, len(optimizers)))
        
        # Plot data for each optimizer
        for j, optimizer in enumerate(optimizers):
            # Filter data for this optimizer
            opt_data = model_data[model_data['optimizer'] == optimizer]
            
            if not opt_data.empty:
                # Extract values for each metric
                values = []
                for metric in metrics:
                    if metric in opt_data.columns:
                        # Normalize the value to 0-1 range for the radar chart
                        min_val = model_data[metric].min()
                        max_val = model_data[metric].max()
                        if max_val > min_val:
                            # For metrics where lower is better (like loss), invert the normalization
                            if 'loss' in metric or 'time' in metric:
                                val = 1 - (opt_data[metric].values[0] - min_val) / (max_val - min_val)
                            else:
                                val = (opt_data[metric].values[0] - min_val) / (max_val - min_val)
                        else:
                            val = 0.5  # Default if all values are the same
                        values.append(val)
                    else:
                        values.append(0)  # Default if metric doesn't exist
                
                values += values[:1]  # Close the loop
                
                # Plot the values
                ax.plot(angles, values, linewidth=2, linestyle='solid', color=colors[j], label=optimizer)
                ax.fill(angles, values, color=colors[j], alpha=0.1)
        
        # Set title and legend
        ax.set_title(f"{model} - Optimizer Performance Comparison")
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot learning curves
if not all_metrics.empty:
    plot_learning_curves(all_metrics, 'val_loss')
    plot_learning_curves(all_metrics, 'val_accuracy')
else:
    print("No metrics data available for plotting learning curves.")

# Create heatmaps for different metrics
if not summary_metrics.empty:
    create_heatmap(summary_metrics, 'min_val_loss', 'Minimum Validation Loss by Model and Optimizer')
    create_heatmap(summary_metrics, 'max_val_accuracy', 'Maximum Validation Accuracy by Model and Optimizer')
    create_heatmap(summary_metrics, 'convergence_epoch', 'Convergence Epoch by Model and Optimizer')
    create_heatmap(summary_metrics, 'training_time', 'Total Training Time (s) by Model and Optimizer')
else:
    print("No summary metrics available for creating heatmaps.")

# Create radar charts
if not summary_metrics.empty:
    radar_metrics = ['min_val_loss', 'max_val_accuracy', 'convergence_epoch', 'training_time']
    create_radar_chart(summary_metrics, selected_optimizers, radar_metrics)
else:
    print("No summary metrics available for creating radar charts.")

## Optimizer Comparison

Let's compare the optimizers more deeply with some additional analyses.

In [ ]:
def compare_optimization_paths(metrics_df, model_name):
    """
    Compares optimization paths in loss/accuracy space for different optimizers
    
    Args:
        metrics_df (pandas.DataFrame): DataFrame with training metrics
        model_name (str): Model to analyze
    """
    if metrics_df.empty:
        print("No metrics data available for optimization path comparison.")
        return
    
    # Filter data for the specified model
    model_data = metrics_df[metrics_df['model'] == model_name]
    
    if model_data.empty:
        print(f"No data available for model {model_name}")
        return
    
    # Get unique optimizers
    optimizers = model_data['optimizer'].unique()
    
    # Colors for different optimizers
    colors = plt.cm.tab10(np.linspace(0, 1, len(optimizers)))
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    for i, optimizer in enumerate(optimizers):
        # Filter data for this optimizer
        opt_data = model_data[model_data['optimizer'] == optimizer]
        
        if not opt_data.empty:
            # Sort by epoch
            opt_data = opt_data.sort_values('epoch')
            
            # Plot the optimization path
            plt.plot(opt_data['val_loss'], opt_data['val_accuracy'], 
                     'o-', label=optimizer, color=colors[i], markersize=8)
            
            # Add arrows to show direction
            for j in range(len(opt_data) - 1):
                plt.annotate('', 
                             xy=(opt_data['val_loss'].iloc[j+1], opt_data['val_accuracy'].iloc[j+1]), 
                             xytext=(opt_data['val_loss'].iloc[j], opt_data['val_accuracy'].iloc[j]),
                             arrowprops=dict(arrowstyle='->', color=colors[i], lw=1.5))
            
            # Add epoch labels for some points
            for j in [0, len(opt_data)//2, len(opt_data)-1]:
                if j < len(opt_data):
                    plt.annotate(f"Epoch {opt_data['epoch'].iloc[j]}", 
                                xy=(opt_data['val_loss'].iloc[j], opt_data['val_accuracy'].iloc[j]),
                                xytext=(10, 0), textcoords='offset points',
                                fontsize=8, color=colors[i])
    
    # Set labels and title
    plt.xlabel('Validation Loss')
    plt.ylabel('Validation Accuracy')
    plt.title(f'Optimization Paths for {model_name} with Different Optimizers')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title="Optimizer")
    
    # Invert x-axis since we want to minimize loss
    plt.gca().invert_xaxis()
    
    plt.tight_layout()
    plt.show()

def analyze_convergence_rates(metrics_df):
    """
    Analyzes convergence rates for different optimizers
    
    Args:
        metrics_df (pandas.DataFrame): DataFrame with training metrics
    """
    if metrics_df.empty:
        print("No metrics data available for convergence rate analysis.")
        return
    
    # Group by model and optimizer
    grouped = metrics_df.groupby(['model', 'optimizer'])
    
    # Calculate the epoch at which the model reached 90% of its best performance
    convergence_data = []
    
    for (model, optimizer), group in grouped:
        # Sort by epoch
        group = group.sort_values('epoch')
        
        # Calculate best val_loss and corresponding epoch
        best_val_loss = group['val_loss'].min()
        best_epoch = group[group['val_loss'] == best_val_loss]['epoch'].iloc[0]
        
        # Calculate epoch at which model reached 90% of best val_loss
        # For loss, we want to find when it's within 110% of the best (since lower is better)
        convergence_threshold = best_val_loss * 1.1
        convergence_epoch = group[group['val_loss'] <= convergence_threshold]['epoch'].min()
        
        convergence_data.append({
            'model': model,
            'optimizer': optimizer,
            'best_val_loss': best_val_loss,
            'best_epoch': best_epoch,
            'convergence_epoch': convergence_epoch,
            'convergence_speed': convergence_epoch / best_epoch if best_epoch > 0 else float('inf')
        })
    
    convergence_df = pd.DataFrame(convergence_data)
    
    # Create the plot
    plt.figure(figsize=(14, 10))
    
    # Get unique models and optimizers
    models = convergence_df['model'].unique()
    optimizers = convergence_df['optimizer'].unique()
    
    # Set up bar positions
    bar_width = 0.15
    index = np.arange(len(models))
    
    # Plot bars for each optimizer
    for i, optimizer in enumerate(optimizers):
        # Filter data for this optimizer
        opt_data = convergence_df[convergence_df['optimizer'] == optimizer]
        
        # Extract convergence epochs for each model
        convergence_epochs = []
        for model in models:
            model_opt_data = opt_data[opt_data['model'] == model]
            if not model_opt_data.empty:
                convergence_epochs.append(model_opt_data['convergence_epoch'].iloc[0])
            else:
                convergence_epochs.append(0)
        
        # Plot the bars
        plt.bar(index + i * bar_width, convergence_epochs, bar_width, 
                label=optimizer, alpha=0.8)
    
    # Set labels and title
    plt.xlabel('Model Architecture')
    plt.ylabel('Convergence Epoch')
    plt.title('Convergence Speed by Model and Optimizer')
    plt.xticks(index + bar_width * (len(optimizers) - 1) / 2, models)
    plt.legend(title="Optimizer")
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Display the convergence data as a table
    display(convergence_df.sort_values(['model', 'convergence_epoch']))
    
    return convergence_df

In [ ]:
# Compare optimization paths for each model
if not all_metrics.empty:
    for model in selected_models:
        compare_optimization_paths(all_metrics, model)
else:
    print("No metrics data available for comparing optimization paths.")

# Analyze convergence rates
if not all_metrics.empty:
    convergence_df = analyze_convergence_rates(all_metrics)
else:
    print("No metrics data available for analyzing convergence rates.")

## Best Optimizer Analysis

Let's determine the best optimizer for each model based on different metrics.

In [ ]:
def find_best_optimizer(summary_df, metric_col, is_lower_better=True):
    """
    Finds the best optimizer for each model based on a specific metric
    
    Args:
        summary_df (pandas.DataFrame): DataFrame with summary metrics
        metric_col (str): Column name for the metric to use
        is_lower_better (bool): Whether a lower value is better (True for loss, False for accuracy)
        
    Returns:
        pandas.DataFrame: DataFrame with best optimizer for each model
    """
    if summary_df.empty:
        print("No summary data available for finding best optimizer.")
        return pd.DataFrame()
    
    # Group by model
    grouped = summary_df.groupby('model')
    
    best_optimizers = []
    
    for model, group in grouped:
        # Find the best optimizer based on the metric
        if is_lower_better:
            best_row = group.loc[group[metric_col].idxmin()]
        else:
            best_row = group.loc[group[metric_col].idxmax()]
        
        best_optimizers.append({
            'model': model,
            'best_optimizer': best_row['optimizer'],
            f'best_{metric_col}': best_row[metric_col]
        })
    
    return pd.DataFrame(best_optimizers)

def analyze_overall_performance(summary_df):
    """
    Analyzes overall performance of optimizers across multiple metrics
    
    Args:
        summary_df (pandas.DataFrame): DataFrame with summary metrics
        
    Returns:
        pandas.DataFrame: DataFrame with overall performance rankings
    """
    if summary_df.empty:
        print("No summary data available for analyzing overall performance.")
        return pd.DataFrame()
    
    # Define metrics and whether lower is better for each
    metric_configs = {
        'min_val_loss': True,      # Lower is better
        'max_val_accuracy': False, # Higher is better
        'convergence_epoch': True, # Lower is better
        'training_time': True      # Lower is better
    }
    
    # Get unique models and optimizers
    models = summary_df['model'].unique()
    optimizers = summary_df['optimizer'].unique()
    
    # Create a DataFrame to store rankings
    rankings = pd.DataFrame(index=optimizers, columns=['overall_score', 'win_count'])
    rankings['overall_score'] = 0
    rankings['win_count'] = 0
    
    # Calculate scores for each optimizer
    for metric, is_lower_better in metric_configs.items():
        # Find best optimizer for each model
        best_opts = find_best_optimizer(summary_df, metric, is_lower_better)
        
        # Update win count
        for _, row in best_opts.iterrows():
            rankings.at[row['best_optimizer'], 'win_count'] += 1
        
        # Calculate normalized scores for each optimizer
        for model in models:
            model_data = summary_df[summary_df['model'] == model]
            
            if not model_data.empty:
                # Get min and max values for this metric across optimizers
                min_val = model_data[metric].min()
                max_val = model_data[metric].max()
                
                if max_val > min_val:  # Avoid division by zero
                    # Calculate normalized scores
                    for optimizer in optimizers:
                        opt_data = model_data[model_data['optimizer'] == optimizer]
                        
                        if not opt_data.empty:
                            value = opt_data[metric].iloc[0]
                            
                            # Normalize to 0-1 where 1 is best
                            if is_lower_better:
                                score = 1 - (value - min_val) / (max_val - min_val)
                            else:
                                score = (value - min_val) / (max_val - min_val)
                            
                            # Update overall score
                            rankings.at[optimizer, 'overall_score'] += score
    
    # Sort by overall score (descending)
    rankings = rankings.sort_values('overall_score', ascending=False)
    
    return rankings

In [ ]:
# Find best optimizer for different metrics
if not summary_metrics.empty:
    print("Best optimizer by validation loss:")
    display(find_best_optimizer(summary_metrics, 'min_val_loss', True))
    
    print("\nBest optimizer by validation accuracy:")
    display(find_best_optimizer(summary_metrics, 'max_val_accuracy', False))
    
    print("\nBest optimizer by convergence speed:")
    display(find_best_optimizer(summary_metrics, 'convergence_epoch', True))
    
    print("\nBest optimizer by training time:")
    display(find_best_optimizer(summary_metrics, 'training_time', True))
else:
    print("No summary metrics available for finding best optimizer.")

# Analyze overall performance
if not summary_metrics.empty:
    overall_rankings = analyze_overall_performance(summary_metrics)
    
    print("Overall optimizer rankings:")
    display(overall_rankings)
    
    # Visualize the rankings
    plt.figure(figsize=(10, 6))
    sns.barplot(y=overall_rankings.index, x='overall_score', data=overall_rankings)
    plt.title('Overall Optimizer Performance')
    plt.xlabel('Performance Score (higher is better)')
    plt.ylabel('Optimizer')
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # Show win count
    plt.figure(figsize=(10, 6))
    sns.barplot(y=overall_rankings.index, x='win_count', data=overall_rankings)
    plt.title('Optimizer Win Count (Best in at least one metric)')
    plt.xlabel('Win Count')
    plt.ylabel('Optimizer')
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("No summary metrics available for analyzing overall performance.")

## Conclusions and Recommendations

Based on our analysis, we can draw the following conclusions about the optimizers and their performance across different model architectures:

### Summary of Findings

1. **Best Overall Optimizer**: Based on the combined metrics (validation loss, accuracy, convergence speed, and training time), [fill in based on results] has shown the best overall performance across all model architectures.

2. **Model-Specific Recommendations**:
   - For **Advanced** model: [fill based on results]
   - For **Residual** model: [fill based on results]
   - For **Best** model: [fill based on results]

3. **Performance Characteristics**:
   - **Convergence Speed**: [fill based on results]
   - **Final Accuracy**: [fill based on results]
   - **Training Efficiency**: [fill based on results]

4. **Trade-offs**:
   - Some optimizers show faster convergence but slightly worse final accuracy.
   - Others take longer to converge but achieve better final results.
   - Training time varies significantly between optimizers and models.

### Recommendations

Based on our analysis, we recommend:

1. **Production Usage**: For production deployments, use [fill based on results] as it provides the best balance of accuracy and efficiency.

2. **Resource-Constrained Environments**: If training time is a significant concern, [fill based on results] provides the fastest convergence with acceptable performance.

3. **Maximum Accuracy**: When accuracy is the primary goal and training time is not a concern, use [fill based on results].

4. **Future Work**:
   - Explore learning rate schedules for each optimizer to potentially improve performance
   - Test custom parameter settings for the top-performing optimizers
   - Investigate combinations of optimizers at different training stages
   - Experiment with more advanced optimizers like LAMB, AdaBelief, or NovoGrad

In [ ]:
# Save the results for future reference
if not summary_metrics.empty:
    # Create a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create output directory if it doesn't exist
    output_dir = Path("optimizer_results")
    output_dir.mkdir(exist_ok=True)
    
    # Save the summary metrics
    summary_metrics.to_csv(output_dir / f"optimizer_comparison_summary_{timestamp}.csv", index=False)
    
    # Save the overall rankings
    if 'overall_rankings' in locals():
        overall_rankings.to_csv(output_dir / f"optimizer_rankings_{timestamp}.csv")
        
    print(f"Results saved with timestamp {timestamp} in {output_dir}")
else:
    print("No results to save.")

# Optimizer Evaluation Across Model Architectures

This notebook provides a comprehensive analysis of different optimizers across various model architectures for our machine learning project. We'll evaluate performance metrics, convergence rates, and overall model quality.

## Table of Contents

1. [Setup and Configuration](#setup-and-configuration)
2. [Model Architectures Overview](#model-architectures-overview)
3. [Optimizer Definitions](#optimizer-definitions)
4. [Training Automation](#training-automation)
5. [Performance Visualization](#performance-visualization)
6. [Optimizer Comparison](#optimizer-comparison)
7. [Best Optimizer Analysis](#best-optimizer-analysis)
8. [Conclusions and Recommendations](#conclusions-and-recommendations)

## Setup and Configuration

Let's import the necessary libraries and set up our environment for the optimizer evaluation experiments. We'll configure paths, import required modules, and define visualization settings.

In [ ]:
import os
import sys
import json
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML

# Add parent directory to path to import project modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Set plotting style
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Configure matplotlib for better visualization
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.dpi'] = 150

# Configuration parameters
PROJECT_ROOT = Path(os.path.abspath(os.path.join(os.getcwd(), '../..')))
RESULTS_DIR = PROJECT_ROOT / 'logs'
MAIN_SCRIPT = PROJECT_ROOT / 'main.py'

# Define models and optimizers to evaluate
MODELS = ['Advanced', 'Residual', 'Best']
OPTIMIZERS = ['Adam', 'ImprovedAdam', 'Nadam', 'RMSprop', 'SGD', 'Adadelta', 'Adagrad', 'Adamax']
EPOCHS = 15  # Use fewer epochs for faster evaluation
BATCH_SIZE = 32

print(f"Project Root: {PROJECT_ROOT}")
print(f"Results Directory: {RESULTS_DIR}")
print(f"Models to evaluate: {MODELS}")
print(f"Optimizers to evaluate: {OPTIMIZERS}")

## Model Architectures Overview

In this section, we'll examine the different model architectures available in our project. We'll load the architectures from our code base and provide a summary of each.

In [ ]:
def train_model(model_type, optimizer, epochs=50, features='medium'):
    """Train a model with the specified optimizer and return performance metrics"""
    print(f"Training {model_type} model with {optimizer} optimizer...")
    
    # Create a timestamp for this run
    start_time = time.time()
    
    # Construct the command to run main.py with appropriate arguments
    cmd = [
        "python", "../../main.py",
        "--model", model_type,
        "--features", features,
        "--loss", "mae",  # Using MAE as standard loss function
        "--epochs", str(epochs),
        "--batch_size", "64",
        "--learning_rate", "0.001"
    ]
    
    # Add optimizer parameter if it's not Adam (which is the default)
    if optimizer != "adam":
        cmd.extend(["--optimizer", optimizer])
    
    print(f"Running command: {' '.join(cmd)}")
    
    # Run the training process
    try:
        process = subprocess.run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        
        # Calculate training time
        training_time = time.time() - start_time
        
        # Check if process was successful
        if process.returncode != 0:
            print(f"Error running command. Exit code: {process.returncode}")
            print(f"Error output: {process.stderr}")
            return None
            
    except Exception as e:
        print(f"Exception running command: {str(e)}")
        return None
    
    # Extract the log directory from the output
    log_dir = None
    for line in process.stdout.split('\n'):
        if "TensorBoard logs saved to:" in line:
            log_dir = line.split("TensorBoard logs saved to:")[1].strip()
            break
    
    if not log_dir:
        print(f"Warning: Could not find log directory in process output")
        
        # Try to find the most recent log directory for this model
        log_base = Path("../../logs/experiments")
        potential_dirs = list(log_base.glob(f"{model_type.capitalize()}Model*"))
        if potential_dirs:
            # Get most recent by sorting
            log_dir = str(sorted(potential_dirs, key=os.path.getmtime, reverse=True)[0])
            print(f"Using most recent log directory: {log_dir}")
        else:
            print("Could not find any log directory")
            return None
    
    # Find metrics file
    metrics_file = None
    if log_dir:
        # Look for final_metrics.json
        metrics_path = Path(log_dir) / "final_metrics.json"
        if metrics_path.exists():
            metrics_file = metrics_path
        else:
            # Try to find metrics in CSV files
            csv_files = list(Path("../../logs/metrics").glob(f"{model_type.capitalize()}Model*.csv"))
            if csv_files:
                most_recent = sorted(csv_files, key=os.path.getmtime, reverse=True)[0]
                print(f"Using metrics from CSV: {most_recent}")
                # Read the last row of the CSV file
                metrics_df = pd.read_csv(most_recent)
                if not metrics_df.empty:
                    # Extract the final epoch metrics
                    last_row = metrics_df.iloc[-1].to_dict()
                    # Create a synthetic metrics object
                    metrics = {
                        "val_loss": last_row.get("val_loss"),
                        "val_mae": last_row.get("val_mae", last_row.get("val_mean_absolute_error")),
                        "val_mse": last_row.get("val_mse", last_row.get("val_mean_squared_error")),
                        "training_time": training_time,
                        "model_name": f"{model_type.capitalize()}Model",
                        "optimizer": optimizer
                    }
                    return metrics
    
    if not metrics_file:
        print(f"Warning: No metrics file found for {model_type} with {optimizer}")
        return None
        
    # Load and return the metrics
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    # Add training time to metrics
    metrics["training_time"] = training_time
    metrics["model_name"] = f"{model_type.capitalize()}Model"
    metrics["optimizer"] = optimizer
    
    print(f"Successfully trained {model_type} with {optimizer} in {training_time:.1f}s")
    return metrics

def extract_all_optimizer_metrics():
    """Try to extract metrics from existing model files without retraining"""
    results = []
    export_dir = Path("../../exports")
    
    # Look for model files with optimizer names in them
    for model_file in export_dir.glob("*.keras"):
        file_name = model_file.name
        
        # Extract model type and optimizer
        for model in MODELS:
            model_cap = model.capitalize()
            if model_cap in file_name:
                for opt in OPTIMIZERS:
                    opt_cap = opt.capitalize()
                    if opt_cap in file_name:
                        print(f"Found existing model: {file_name}")
                        
                        # Try to load the model metrics from logs
                        metrics = {
                            "model": model,
                            "optimizer": opt,
                            "file_name": file_name
                        }
                        results.append(metrics)
    
    return results

## 3. Run Optimizer Evaluation

Now we'll execute the evaluation by training each model with each optimizer:

In [ ]:
# Function to train all combinations and gather results
def run_full_evaluation(models=MODELS, optimizers=OPTIMIZERS, epochs=EPOCHS):
    all_results = []
    
    # Create a grid of model and optimizer combinations
    total_combinations = len(models) * len(optimizers)
    completed = 0
    
    for model in models:
        for opt in optimizers:
            # Show progress
            completed += 1
            print(f"\n[{completed}/{total_combinations}] Evaluating {model} with {opt}...")
            
            try:
                metrics = train_model(model, opt, epochs=epochs)
                if metrics:
                    # Extract key metrics
                    result = {
                        'model': model,
                        'optimizer': opt,
                        'val_loss': metrics.get('val_loss'),
                        'val_mae': metrics.get('val_mae'),
                        'val_mse': metrics.get('val_mse'),
                        'correlation': metrics.get('correlation', metrics.get('val_correlation')),
                        'training_time': metrics.get('training_time')
                    }
                    all_results.append(result)
                    
                    # Save intermediate results in case of failure
                    pd.DataFrame(all_results).to_csv(results_file, index=False)
                    print(f"✅ Completed {model} with {opt}")
                else:
                    print(f"❌ Failed to get metrics for {model} with {opt}")
            except Exception as e:
                print(f"❌ Error training {model} with {opt}: {str(e)}")
    
    return all_results

# Comment out the line below if you want to run the evaluation
# Otherwise, we'll load previously saved results in the next cell
# all_results = run_full_evaluation()

## 4. Data Analysis and Visualization

Now let's analyze and visualize the results to compare optimizer performance:

In [ ]:
# Load saved results or use the ones we just generated
try:
    # Try to load the most recent results file if it exists
    result_files = list(results_dir.glob("optimizer_comparison_*.csv"))
    if result_files and not 'all_results' in locals():
        most_recent = sorted(result_files, key=os.path.getmtime, reverse=True)[0]
        print(f"Loading existing results from {most_recent}")
        results_df = pd.read_csv(most_recent)
    else:
        # Use the results we just generated
        results_df = pd.DataFrame(all_results)
        results_df.to_csv(results_file, index=False)
        print(f"Results saved to {results_file}")
except Exception as e:
    print(f"Error loading results: {str(e)}")
    print("You might need to run the evaluation first by uncommenting the run_full_evaluation() call")
    # Create a dummy DataFrame for demonstration
    results_df = pd.DataFrame({
        'model': ['base', 'base', 'advanced', 'advanced', 'residual', 'residual'],
        'optimizer': ['adam', 'sgd', 'adam', 'sgd', 'adam', 'sgd'],
        'val_loss': [0.52, 0.56, 0.48, 0.51, 0.45, 0.47],
        'val_mae': [0.52, 0.56, 0.48, 0.51, 0.45, 0.47],
        'correlation': [0.65, 0.60, 0.70, 0.68, 0.72, 0.71],
        'training_time': [45, 50, 60, 65, 75, 80]
    })

# Display the raw results
print("Raw evaluation results:")
display(results_df)

# Create pivot tables for each metric
metrics = ['val_loss', 'val_mae', 'correlation', 'training_time']
pivot_tables = {}

for metric in metrics:
    if metric in results_df.columns:
        pivot_tables[metric] = results_df.pivot(index='model', columns='optimizer', values=metric)
    
# Display the pivot tables
for metric, pivot in pivot_tables.items():
    print(f"\n{metric.upper()} by model and optimizer:")
    display(pivot)

## 5. Visualization: Heatmaps

Let's visualize the results using heatmaps to easily identify the best optimizer for each model:

In [ ]:
# Function to visualize the results as heatmap
def plot_metric_heatmap(data, title, cmap='RdYlGn_r', fmt='.4f'):
    if data is None or data.empty:
        print(f"No data available for {title} heatmap")
        return
        
    plt.figure(figsize=(16, 6))
    
    # For correlation metrics, higher is better so reverse the colormap
    if 'correlation' in title.lower():
        cmap = 'RdYlGn'
    
    # Create the heatmap
    ax = sns.heatmap(
        data, 
        annot=True, 
        cmap=cmap, 
        fmt=fmt, 
        linewidths=.5,
        annot_kws={"size": 10}
    )
    
    # Set title and labels
    plt.title(f"{title} by Model and Optimizer", fontsize=16)
    plt.ylabel('Model Architecture', fontsize=12)
    plt.xlabel('Optimizer', fontsize=12)
    
    # Rotate x-axis labels
    plt.xticks(rotation=45, ha='right')
    
    # Save and show
    plt.tight_layout()
    plt.savefig(results_dir / f"{title.lower().replace(' ', '_')}.png")
    plt.show()

# Create heatmaps for each metric
if 'val_loss' in pivot_tables:
    plot_metric_heatmap(pivot_tables['val_loss'], "Validation Loss", fmt='.4f')
    
if 'val_mae' in pivot_tables:
    plot_metric_heatmap(pivot_tables['val_mae'], "Mean Absolute Error", fmt='.4f')
    
if 'correlation' in pivot_tables:
    plot_metric_heatmap(pivot_tables['correlation'], "Correlation Coefficient", fmt='.4f')
    
if 'training_time' in pivot_tables:
    plot_metric_heatmap(pivot_tables['training_time'], "Training Time (seconds)", fmt='.1f')

## 6. Visualization: Radar Charts

Let's create radar charts to compare optimizers across all model architectures:

In [ ]:
def plot_radar_chart(df, metric='correlation'):
    if metric not in df.columns:
        print(f"Metric '{metric}' not found in results dataframe")
        return
        
    # Group by optimizer and calculate mean for the metric
    optimizer_means = df.groupby('optimizer')[metric].mean().reset_index()
    
    # Sort by performance
    ascending = not (metric == 'correlation')  # For correlation, higher is better
    optimizer_means = optimizer_means.sort_values(by=metric, ascending=ascending)
    
    # Prepare the radar chart
    labels = optimizer_means['optimizer']
    values = optimizer_means[metric].values
    
    # Number of variables
    N = len(labels)
    
    # Create angle for each variable
    angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
    
    # Close the loop
    angles += angles[:1]
    
    # Values need to be repeated to close the loop
    values = np.concatenate((values, [values[0]]))
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    ax.plot(angles, values, linewidth=2, linestyle='solid')
    ax.fill(angles, values, alpha=0.25)
    
    # Add labels
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    
    # Set y-axis limits based on the metric
    if metric == 'correlation':
        ax.set_ylim(min(values) - 0.05, max(values) + 0.05)
    
    plt.title(f"Average {metric.replace('_', ' ').title()} by Optimizer", size=15)
    plt.tight_layout()
    plt.savefig(results_dir / f"radar_{metric}.png")
    plt.show()
    
    # Also print the ranking table
    print(f"Optimizer ranking by average {metric}:")
    display(optimizer_means)

# Generate radar charts for different metrics
for metric in ['correlation', 'val_loss', 'val_mae', 'training_time']:
    if metric in results_df.columns:
        plot_radar_chart(results_df, metric)

## 7. Best Optimizer For Each Model

Finally, let's determine the best optimizer for each model architecture:

In [ ]:
def find_best_optimizer(df, metric='correlation', is_higher_better=True):
    """Find the best optimizer for each model based on the specified metric"""
    if metric not in df.columns:
        print(f"Metric '{metric}' not found in results dataframe")
        return None
        
    best_optimizers = []
    
    # Group by model
    for model, group in df.groupby('model'):
        if is_higher_better:
            best_row = group.loc[group[metric].idxmax()]
        else:
            best_row = group.loc[group[metric].idxmin()]
            
        best_optimizers.append({
            'model': model,
            'best_optimizer': best_row['optimizer'],
            f'best_{metric}': best_row[metric]
        })
        
    return pd.DataFrame(best_optimizers)

# Find best optimizer by correlation (higher is better)
best_by_correlation = find_best_optimizer(results_df, 'correlation', is_higher_better=True)
print("Best optimizers by correlation (higher is better):")
display(best_by_correlation)

# Find best optimizer by validation loss (lower is better)
if 'val_loss' in results_df.columns:
    best_by_loss = find_best_optimizer(results_df, 'val_loss', is_higher_better=False)
    print("\nBest optimizers by validation loss (lower is better):")
    display(best_by_loss)

# Create a summary visualization
plt.figure(figsize=(12, 8))
    
models = results_df['model'].unique()
for i, model in enumerate(models):
    # Get data for this model
    model_data = results_df[results_df['model'] == model]
    
    # Create subplot for this model
    plt.subplot(len(models), 1, i+1)
    
    if 'correlation' in model_data.columns:
        # Sort by correlation
        sorted_data = model_data.sort_values(by='correlation', ascending=False)
        
        # Plot correlation bars
        bars = plt.barh(
            sorted_data['optimizer'], 
            sorted_data['correlation'], 
            color='skyblue'
        )
        
        # Add value labels to the end of each bar
        for bar in bars:
            width = bar.get_width()
            plt.text(
                width + 0.01, 
                bar.get_y() + bar.get_height()/2, 
                f'{width:.4f}', 
                ha='left', 
                va='center'
            )
            
        plt.xlim(0, max(sorted_data['correlation']) * 1.1)
        plt.title(f"{model.capitalize()} Model - Correlation by Optimizer")
        plt.xlabel("Correlation")
    
plt.tight_layout()
plt.savefig(results_dir / "optimizer_summary.png")
plt.show()

# Create recommendations based on findings
print("\n=== OPTIMIZER RECOMMENDATIONS ===")
print("Based on the evaluation results, here are the recommended optimizers:")
for model in models:
    best_opt = best_by_correlation[best_by_correlation['model'] == model]['best_optimizer'].values[0]
    best_corr = best_by_correlation[best_by_correlation['model'] == model]['best_correlation'].values[0]
    print(f"- For {model.capitalize()} model: Use {best_opt} optimizer (correlation = {best_corr:.4f})")

## 8. Conclusion

In this notebook, we have:
1. Set up a comprehensive optimizer evaluation framework
2. Trained multiple models with various optimizers
3. Collected and analyzed performance metrics
4. Visualized the results using heatmaps and radar charts
5. Determined the best optimizer for each model architecture

This analysis provides valuable insights for model selection and hyperparameter tuning in our project.